In [1]:
import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
from scipy.spatial.distance import cosine
from numpy.linalg import norm

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [41]:
filename = "data/all.txt"

words = open(filename, encoding='utf8').read().split()
vocab = list(set(words))
print('ከባንኮክ' in words)

vocabulary_size = len(vocab)
print('Data size', len(words), ' Vocab Size', vocabulary_size)

def build_dataset(words):
    """Process raw inputs into a dataset."""

    dictionary = dict()
    for word in words:
        if word not in dictionary:
            dictionary[word] = len(dictionary)
    data = list()
    for word in words:
        index = dictionary[word]
        data.append(index)
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, dictionary, reversed_dictionary



True
Data size 10193286  Vocab Size 610151


In [8]:
data, dictionary, reverse_dictionary = build_dataset(words)
del words
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Sample data [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] ['ጋዜጠኛ', 'ተመስገን', 'ደሳለኝ', 'በጠበቃው', 'በአቶ', 'አምሐ', 'መኮንን', 'አማካይነት', 'በፌዴራል', 'ጠቅላይ']


In [10]:

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
#     batch_size = batch_size // 2
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(
        maxlen=span)  # pylint: disable=redefined-builtin
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
            
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels


batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=2)
for i in range(len(batch)):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
          reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.


2 ደሳለኝ -> 4 በአቶ
2 ደሳለኝ -> 3 በጠበቃው
3 በጠበቃው -> 5 አምሐ
3 በጠበቃው -> 1 ተመስገን
4 በአቶ -> 2 ደሳለኝ
4 በአቶ -> 6 መኮንን
5 አምሐ -> 3 በጠበቃው
5 አምሐ -> 4 በአቶ


In [22]:

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
np.random.seed(1000)
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

n_words = len(data)
epoches = 10
steps_per_batch = n_words // batch_size


In [12]:
import gensim, logging
import collections

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentenses = open(filename, encoding='utf-8').read().split('*')
sentenses = [s.strip().split() for s in sentenses]
gensim_model = gensim.models.Word2Vec(sentenses, size=embedding_size, iter=1, min_count=1)
print("Gensim loaded")

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Gensim loaded


In [37]:

graph = tf.Graph()

with graph.as_default():

    # Input data.
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/GPU:0'):
        # Look up embeddings for inputs.
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                tf.random_normal([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)    
            print(embed.get_shape())

        # Construct the variables for the NCE loss
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                tf.truncated_normal(
                    [vocabulary_size, embedding_size],
                    stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    # Explanation of the meaning of NCE loss:
    #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=embed,
                num_sampled=num_sampled,
                num_classes=vocabulary_size))

    # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norms = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norms
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()


(128, 128)


In [38]:
epoches = 2
num_steps = steps_per_batch * epoches
log_on = num_steps // (100 // epoches)
with tf.Session(graph=graph) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter('./log', session.graph)

    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                    skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # Define metadata variable.
        run_metadata = tf.RunMetadata()

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
        # Feed metadata variable to session for visualizing the graph in TensorBoard.
        _, summary, loss_val = session.run(
            [optimizer, merged, loss],
            feed_dict=feed_dict,
            run_metadata=run_metadata)
        average_loss += loss_val

        # Add returned summaries to writer in each step.
        writer.add_summary(summary, step)
        # Add metadata to visualize the graph for the last run.
        if step == (num_steps - 1):
            writer.add_run_metadata(run_metadata, 'step%d' % step)

        if step % log_on == 0:
            if step > 0:
                average_loss /= log_on
            # The average loss is an estimate of the loss over the last 2000 batches.
            log_text = "Progress: {0:.2f}% Averlage loss: {1:.2f}".format((step * 100 / steps_per_batch), average_loss)
            print(log_text)
            average_loss = 0

#         # Note that this is expensive (~20% slowdown if computed every 500 steps)
#         if step % 10000 == 0:
#             sim = similarity.eval()
#             for i in xrange(valid_size):
#                 valid_word = reverse_dictionary[valid_examples[i]]
#                 top_k = 8  # number of nearest neighbors
#                 nearest = (-sim[i, :]).argsort()[1:top_k + 1]
#                 log_str = 'Nearest to %s:' % valid_word
#                 for k in xrange(top_k):
#                     close_word = reverse_dictionary[nearest[k]]
#                     log_str = '%s %s,' % (log_str, close_word)
#                 print(log_str)
    final_embeddings = normalized_embeddings.eval()
    
    # Write corresponding labels for the embeddings.
    with open('./log/metadata.tsv', 'w', encoding='utf8') as f:
        for i in xrange(vocabulary_size):
            f.write(reverse_dictionary[i] + '\n')

    # Save the model for checkpoints.
    saver.save(session, os.path.join('./log', 'model.ckpt'))

    # Create a configuration for visualizing embeddings with the labels in TensorBoard.
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = os.path.join('./log', 'metadata.tsv')
    projector.visualize_embeddings(writer, config)

writer.close()

Initialized
Progress: 0.00% Averlage loss: 439.06
Progress: 1.00% Averlage loss: 279.37
Progress: 2.00% Averlage loss: 229.89
Progress: 3.00% Averlage loss: 213.15
Progress: 4.00% Averlage loss: 194.85
Progress: 5.00% Averlage loss: 183.26
Progress: 6.00% Averlage loss: 172.58
Progress: 7.00% Averlage loss: 162.15
Progress: 8.00% Averlage loss: 167.28
Progress: 9.00% Averlage loss: 158.53
Progress: 10.00% Averlage loss: 151.34
Progress: 11.00% Averlage loss: 141.99
Progress: 11.99% Averlage loss: 137.64
Progress: 12.99% Averlage loss: 133.67
Progress: 13.99% Averlage loss: 127.64
Progress: 14.99% Averlage loss: 124.86
Progress: 15.99% Averlage loss: 121.60
Progress: 16.99% Averlage loss: 115.53
Progress: 17.99% Averlage loss: 112.51
Progress: 18.99% Averlage loss: 112.41
Progress: 19.99% Averlage loss: 106.94
Progress: 20.99% Averlage loss: 105.26
Progress: 21.99% Averlage loss: 100.33
Progress: 22.99% Averlage loss: 99.36
Progress: 23.99% Averlage loss: 97.28
Progress: 24.99% Averlage

In [39]:
for gindex in range(len(gensim_model.wv.index2word)):
    gword = gensim_model.wv.index2word[gindex]
    index = dictionary[gword]
    embedding = final_embeddings[index]
    gensim_model.wv.vectors[gindex] = embedding
print("Loaded in to gensim")

Loaded in to gensim


In [40]:
gensim_model.accuracy('data/syntax.txt')

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.wv.evaluate_word_analogies() instead).
  """Entry point for launching an IPython kernel.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[{'section': 'syntactic evaluation',
  'correct': [],
  'incorrect': [('ትሄዳለች', 'እየሄደች', 'ይበላል', 'እየበላ'),
   ('ትሄዳለች', 'እየሄደች', 'ታደርጋለች', 'እያደረገች'),
   ('ትሄዳለች', 'እየሄደች', 'ይመለከታል', 'እየተመለከተ'),
   ('ትሄዳለች', 'እየሄደች', 'ይሄዳል', 'እየሄደ'),
   ('ትሄዳለች', 'እየሄደች', 'ትላለች', 'እያለች'),
   ('ትሄዳለች', 'እየሄደች', 'ያስባል', 'እያሰበ'),
   ('ትሄዳለች', 'እየሄደች', 'ያወጣል', 'እያወጣ'),
   ('ትሄዳለች', 'እየሄደች', 'ይመስላል', 'እየመሰለ'),
   ('ይበላል', 'እየበላ', 'ትሄዳለች', 'እየሄደች'),
   ('ይበላል', 'እየበላ', 'ታደርጋለች', 'እያደረገች'),
   ('ይበላል', 'እየበላ', 'ይመለከታል', 'እየተመለከተ'),
   ('ይበላል', 'እየበላ', 'ይሄዳል', 'እየሄደ'),
   ('ይበላል', 'እየበላ', 'ትላለች', 'እያለች'),
   ('ይበላል', 'እየበላ', 'ያስባል', 'እያሰበ'),
   ('ይበላል', 'እየበላ', 'ያወጣል', 'እያወጣ'),
   ('ይበላል', 'እየበላ', 'ይመስላል', 'እየመሰለ'),
   ('ታደርጋለች', 'እያደረገች', 'ትሄዳለች', 'እየሄደች'),
   ('ታደርጋለች', 'እያደረገች', 'ይበላል', 'እየበላ'),
   ('ታደርጋለች', 'እያደረገች', 'ይመለከታል', 'እየተመለከተ'),
   ('አድርጋለች', 'ልታደርግ', 'ሄዷል', 'ሊሄድ'),
   ('ታደርጋለች', 'እያደረገች', 'ይሄዳል', 'እየሄደ'),
   ('አድርጋለች', 'ልታደርግ', 'ተቀምጧል', 'ሊቀመጥ'),
   ('አድርጋለች', 'ልታደርግ', 'ብላለች', 'ልትል'),
   ('ታደርጋለች'

In [35]:
gensim_model.similar_by_word('ዝቅተኛ')

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ቱንጋ', 0.7262619733810425),
 ('እንደሚያጠጣት', 0.7166581153869629),
 ('ስላስፈራራው', 0.7128387689590454),
 ('ያጣናን', 0.7123674750328064),
 ('ፊታዉራሪን', 0.7114162445068359),
 ('ቢያሳትምም', 0.7111265659332275),
 ('መሂድ', 0.7107560634613037),
 ('ሳናሱሲ', 0.7092135548591614),
 ('ከጃፓኑ', 0.7065313458442688),
 ('ቢዞሩ', 0.705833911895752)]